In [2]:
import PIL.Image as Image
import gradio as gr
from ultralytics import YOLO
import os
import time
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variables
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables. Please add it to your .env file.")


# Initialize object detection models
tomato_model = YOLO("tomato.pt")
fruit_model = YOLO("fruits.pt")

# Global variable to control response generation
stop_generation = False

# Initialize chatbot components
llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
prompt = ChatPromptTemplate.from_template(
    """Answer the questions based on the provided context only. Please provide the most accurate response based on the question <context> {context} </context> Questions:{input} Respond in the same language as the question."""
)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
loader = PyPDFLoader("DOC.pdf")
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
final_documents = text_splitter.split_documents(docs)
doc_texts = [doc.page_content for doc in final_documents]
embeddings_result = embeddings.embed_documents(doc_texts)
if embeddings_result:
    vectors = FAISS.from_documents(final_documents, embeddings)
else:
    raise ValueError("Failed to generate embeddings. Please check your input documents or try a different embedding model.")

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

from transformers import M2M100Tokenizer

# Initialize translation model and tokenizer
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


def translate_text(text, target_language):
    if target_language == "Tamil":
        target_lang_code = "ta"
    elif target_language == "Hindi":
        target_lang_code = "hi"
    else:
        return "Unsupported language"

    # Tokenize input text
    m2m_tokenizer.src_lang = "en"
    encoded_text = m2m_tokenizer(text, return_tensors="pt")

    # Generate translation
    generated_tokens = m2m_model.generate(**encoded_text, forced_bos_token_id=m2m_tokenizer.get_lang_id(target_lang_code))

    # Decode the generated tokens
    translated_text = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

def predict_image(img, conf_threshold, iou_threshold, model_choice):
    model = tomato_model if model_choice == "Tomato Leaf Disease" else fruit_model
    results = model.predict(source=img, conf=conf_threshold, iou=iou_threshold, show_labels=True, show_conf=True, imgsz=640)
    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])
    return im

def add_message(history, message):
    history.append((message, None))
    return history, gr.Textbox(value="", interactive=True)

def bot(history):
    global stop_generation
    stop_generation = False
    
    if not history:
        history.append(("Hello! How can I assist you today?", None))
        yield history
        return

    message = history[-1][0]
    try:
        response = retrieval_chain.invoke({'input': message})['answer']
    except Exception as e:
        response = f"I'm sorry, but I encountered an error: {str(e)}"
    
    history[-1] = (history[-1][0], "")
    for character in response:
        if stop_generation:
            break
        history[-1] = (history[-1][0], history[-1][1] + character)
        time.sleep(0.01)  # Reduced delay for faster response
        yield history

def translate(history, target_language):
    if not history:
        return [("No message to translate.", None)]
    
    original_text = history[-1][1]
    if not original_text:
        return history + [("No text to translate.", None)]
    
    try:
        translated_text = translate_text(original_text, target_language)
        history.append((f"Translated to {target_language}:", translated_text))
    except Exception as e:
        history.append((f"Translation error:", f"I'm sorry, but I encountered an error while translating: {str(e)}"))
    
    return history

def stop_response():
    global stop_generation
    stop_generation = True

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=2):
            model_input = gr.Image(type="pil", label="Upload Image")
            model_choice = gr.Radio(["Tomato Leaf Disease", "Fruit Disease"], label="Choose Model", value="Tomato Leaf Disease")
            conf_threshold = gr.Slider(minimum=0, maximum=1, value=0.25, label="Confidence threshold")
            iou_threshold = gr.Slider(minimum=0, maximum=1, value=0.45, label="IoU threshold")
            model_output = gr.Image(type="pil", label="Result")

            model_btn = gr.Button("Detect Disease")
            model_btn.click(predict_image, inputs=[model_input, conf_threshold, iou_threshold, model_choice], outputs=model_output)

        with gr.Column(scale=1):
            chatbot = gr.Chatbot([], elem_id="chatbot")
            msg = gr.Textbox(label="Message")
            clear = gr.Button("Clear")

            chat_msg = msg.submit(add_message, [chatbot, msg], [chatbot, msg]).then(
                bot, chatbot, chatbot
            )

            clear.click(lambda: [], None, chatbot, queue=False)

            with gr.Row():
                tamil_btn = gr.Button("Translate to Tamil")
                hindi_btn = gr.Button("Translate to Hindi")

            tamil_btn.click(translate, [chatbot, gr.Textbox(value="Tamil", visible=False)], chatbot)
            hindi_btn.click(translate, [chatbot, gr.Textbox(value="Hindi", visible=False)], chatbot)

            stop_btn = gr.Button("Stop Generation")
            stop_btn.click(stop_response)

    demo.queue()

if __name__ == "__main__":
    demo.launch()

c:\Users\Ramnivash\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /facebook/m2m100_418M/resolve/main/model.safetensors (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000207390AA180>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: fcf04f24-c460-4492-8343-9bd31ba982ab)')